In [1]:
# import functions from utils and porteval
import sys
import os
import pandas as pd

from portopt.portfolio import Portfolio

from portopt.config import load_config
from portopt.utils import write_table, write_weights

from portopt.rebalance import PortfolioRebalancer

In [ ]:
# required file paths
portfolio_dir = "../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")

# to reduce amount of price data that needs to be downloaded, we only use the first holdings file
# comment this out to use all holdings files
num_files = 3
holdings_files = [
    os.path.join(holdings_files, f) 
    for f in os.listdir(holdings_files) 
    if os.path.isfile(os.path.join(holdings_files, f)) and f.endswith('.csv')
][:num_files]

factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:")
for file in holdings_files:
    print(f"  {file}")

In [3]:
# load config
config = load_config(config_file)
import pprint
#pprint.pprint(config)

In [4]:
# define column formats for write_table function
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Level_5': {'width': 14},
    'Level_6': {'width': 14},
    'Factor': {'width': 24},
    'Weight': {'width': 14, 'decimal': 3, 'type':'%'},
    'Account': {'width': 25, 'align': '<'},
    'Name': {'width': 30, 'align': '<'},
    'Short Name': {'width': 20, 'align': '<'},
    'Institution': {'width': 14},
    'Type': {'width': 14},
    'Category': {'width': 14},
    'Family': {'width': 14},
    'Owner': {'width': 14},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Ticker': {'width': 14},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Total Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'New Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Value Diff': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Original Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'New Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Target Allocation': {'width': 16, 'decimal': 2, 'type':'%'},
    'Original Difference': {'width': 16, 'decimal': 2, 'type':'%'},
    'Target Difference': {'width': 16, 'decimal': 2, 'type':'%'}
}
#column_formats

In [ ]:
# create portfolio object
port = Portfolio(config, factor_weights_file, holdings_files)
port

In [ ]:
factor_weights = port.getFactorWeights()
write_table(factor_weights, column_formats)

In [ ]:
# create a target allocation for the portfolio
# Reduce Alt-Crypto, Alt-Gold, and Equity-US by 5 percentage points
results = port.adjust_factor_allocations(
    source_filter={'Level_0': ['Equity'], 'Level_1': ['US']},
    dest_filter={'Level_0': ['Fixed Income'], 'Level_1': ['Cash']},
    transfer=0.02,  # 15 percentage points
    verbose=True
)
write_table(results, column_formats)

total_orig_alloc = results['Original Allocation'].sum()
total_new_alloc = results['New Allocation'].sum()
print(f"Total original allocation: {total_orig_alloc:,.2%}")
print(f"Total new allocation: {total_new_alloc:,.2%}")

target_factor_allocations = results['New Allocation']
write_table(target_factor_allocations, column_formats)

In [ ]:
# get account tickers and allocation for each ticker
account_ticker_allocations = port.getMetrics('Account', 'Ticker',
                                             metrics=['Allocation'],
                                             portfolio_allocation=True)
write_table(account_ticker_allocations, column_formats)


In [ ]:
# create a PortfolioRebalancer object
rebalancer = PortfolioRebalancer(account_ticker_allocations=account_ticker_allocations,
                                 target_factor_allocations=target_factor_allocations,
                                 factor_weights=factor_weights,
                                 min_ticker_alloc=0.01,
                                 turnover_penalty=0.25,
                                 complexity_penalty=0.25,
                                 account_align_penalty=0.25,
                                 verbose=True)


In [ ]:
import cvxpy as cp

verbose = True

accounts = rebalancer.getAccounts()
test_account = accounts[0]

# get account rebalancer    
account_rebalancer = rebalancer.getAccountRebalancer(test_account)

# get factor weights matrix
factor_weights_df = account_rebalancer.getFactorWeights()
write_weights(factor_weights_df, "Factor Weights Matrix")

# get optimization variables
variables = account_rebalancer.getVariables()
x = variables['x']
print(x)

# define optimized factor allocations
optimized_factor_allocations = factor_weights_df.to_numpy() @ x

# get target factor allocations
target_factor_allocations = account_rebalancer.getTargetFactorAllocations()
write_weights(target_factor_allocations, "Target Factor Allocations")

# Objective: Minimize squared difference + sparsity penalty
objective = cp.Minimize(
    cp.sum_squares(optimized_factor_allocations - target_factor_allocations)
)

# Constraints
constraints = [
    cp.sum(x) == 1,  # Allocations must sum to 100%
    x >= 0,          # No negative allocation
]

# Solve the problem using SCIP solver (supports MIQP)
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP,
              verbose=verbose)

print(f"\nOptimization complete:")
print(f" - Status: {problem.status}")
print(f" - Objective value: {problem.value:.12f}")

# Get new ticker allocations
new_ticker_allocations = pd.Series(
    data = x.value.flatten(),
    index=factor_weights_df.columns,
    name='New Allocation'
)
write_weights(new_ticker_allocations, "New Ticker Allocations")
ticker_allocations_sum = new_ticker_allocations.sum()
print(f" - Ticker allocations sum: {ticker_allocations_sum:,.12%}")

# Get new factor allocations
new_factor_allocations = pd.Series(
    factor_weights_df.to_numpy() @ new_ticker_allocations.to_numpy(),
    index=factor_weights_df.index,
    name='New Allocation'
)
factor_results = pd.DataFrame(
    {
        'Target Allocation': target_factor_allocations,
        'New Allocation': new_factor_allocations
    },
    index=factor_weights_df.index
)
factor_results['Difference'] = factor_results['New Allocation'] - factor_results['Target Allocation']
write_weights(factor_results, "Factor Results")
factor_allocations_sum = new_factor_allocations.sum()
print(f" - Factor allocations sum: {factor_allocations_sum:,.12%}")



In [ ]:
import cvxpy as cp

verbose = True

# Create account ticker allocations (account is 100% of portfolio)
ticker_allocations = {
    ('TestAccount', 'ABCD'): 0.40,
    ('TestAccount', 'EFGH'): 0.25,
    ('TestAccount', 'JKLM'): 0.35
}
ticker_allocations_df = pd.Series(ticker_allocations).to_frame(name='Allocation')
ticker_allocations_df.index.names = ['Account', 'Ticker']
write_weights(ticker_allocations_df, title="Ticker Allocations")

# Create target factor allocations
target_factor_allocations = {
    'Factor1': 0.25,
    'Factor2': 0.35,
    'Factor3': 0.40
}
target_factor_allocations_df = pd.Series(target_factor_allocations, name='Target Allocation')
target_factor_allocations_df.index.name = 'Factor'
target_factor_allocations_df = target_factor_allocations_df.sort_index()
write_weights(target_factor_allocations_df, title="Target Factor Allocations")

# Define factor weights in long format with MultiIndex
factor_weights = {
    ('ABCD', 'Factor1'): 1,
    ('ABCD', 'Factor2'): 0,
    ('ABCD', 'Factor3'): 0,
    ('EFGH', 'Factor1'): 0,
    ('EFGH', 'Factor2'): 1,
    ('EFGH', 'Factor3'): 0,
    ('JKLM', 'Factor1'): 0,
    ('JKLM', 'Factor2'): 0,
    ('JKLM', 'Factor3'): 1
}
factor_weights_df = pd.Series(factor_weights).to_frame(name='Weight')
factor_weights_df.index.names = ['Ticker', 'Factor']
write_weights(factor_weights_df, "Factor Weights Table")

# Create PortfolioRebalancer
port_rebalancer = PortfolioRebalancer(
    account_ticker_allocations=ticker_allocations_df,
    target_factor_allocations=target_factor_allocations_df,
    factor_weights=factor_weights_df,
    verbose=True
)

# Get AccountRebalancer instance
account_rebalancer = port_rebalancer.getAccountRebalancer('TestAccount')

# Run rebalancing
account_rebalancer.rebalance(verbose=False)

# Get results
ticker_results = account_rebalancer.getTickerResults()
from portopt.utils import write_table   
write_table(ticker_results, columns=column_formats, title="Ticker Results")

factor_results = account_rebalancer.getFactorResults()
from portopt.utils import write_table
write_table(factor_results, columns=column_formats, title="Factor Results")




In [ ]:

# Define factor weights matrix
factor_weights = {
    'Factor': ['Factor1', 'Factor2', 'Factor3'],
    'ABCD': [1, 0, 0],
    'EFGH': [0, 1, 0],
    'JKLM': [0, 0, 1]
}
factor_weights_df = pd.DataFrame(factor_weights)
factor_weights_df = factor_weights_df.set_index('Factor').sort_index().sort_index(axis=1)
write_weights(factor_weights_df, "Factor Weights Matrix")

# Define optimization variables
num_funds = factor_weights_df.shape[1] # number of columns
x = cp.Variable(num_funds)  # Allocation to each fund

# Optimized portfolio allocation
optimized_factor_allocations = factor_weights_df.to_numpy() @ x

# Define target factor allocations
target_factor_allocations = pd.Series(
    data=[0.40, 0.25, 0.35],  # Factor1: 40%, Factor2: 25%, Factor3: 35%
    index=factor_weights_df.index,
    name='Target Allocation'
)
write_weights(target_factor_allocations, "Target Factor Allocations")

# Objective: Minimize squared difference + sparsity penalty
objective = cp.Minimize(
    cp.sum_squares(optimized_factor_allocations - target_factor_allocations)
)

# Constraints
constraints = [
    cp.sum(x) == 1,  # Allocations must sum to 100%
    x >= 0,          # No negative allocation
]

# Solve the problem using SCIP solver (supports MIQP)
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP,
              verbose=verbose)

print(f"\nOptimization complete:")
print(f" - Status: {problem.status}")
print(f" - Objective value: {problem.value:.12f}")

# Get new ticker allocations
new_ticker_allocations = pd.Series(
    data = x.value.flatten(),
    index=factor_weights_df.columns,
    name='New Allocation'
)
write_weights(new_ticker_allocations, "New Ticker Allocations")
ticker_allocations_sum = new_ticker_allocations.sum()
print(f" - Ticker allocations sum: {ticker_allocations_sum:,.12%}")

# Get new factor allocations
new_factor_allocations = pd.Series(
    factor_weights_df.to_numpy() @ new_ticker_allocations.to_numpy(),
    index=factor_weights_df.index,
    name='New Allocation'
)
factor_results = pd.DataFrame(
    {
        'Target Allocation': target_factor_allocations,
        'New Allocation': new_factor_allocations
    },
    index=factor_weights_df.index
)
factor_results['Difference'] = factor_results['New Allocation'] - factor_results['Target Allocation']
write_weights(factor_results, "Factor Results")
factor_allocations_sum = new_factor_allocations.sum()
print(f" - Factor allocations sum: {factor_allocations_sum:,.12%}")


In [ ]:
accounts = rebalancer.getAccounts()
test_account = accounts[0]

account_tickers = rebalancer.getAccountTickers(test_account)
print(type(account_tickers))
for ticker in account_tickers:
    print(ticker)

print("\n")

account_ticker_allocations = rebalancer.getAccountOriginalTickerAllocations(test_account)
print(type(account_ticker_allocations))
for ticker, allocation in account_ticker_allocations.items():
    print(f"{ticker}: {allocation}")

print("\n")

account_ticker_results = rebalancer.getAccountTickerResults(test_account)
print(type(account_ticker_results))
write_table(account_ticker_results, column_formats)

print("\n")

account_variables = rebalancer.getAccountVariables(test_account, verbose=True)
print(type(account_variables))
# for variable, value in account_variables.items():
#     print(f"{variable}: {value}")

print("\n")

portfolio_factors = rebalancer.getAccountRebalancer(test_account).getFactors(verbose=True)
print(type(portfolio_factors))
print(portfolio_factors)

account_factor_allocations = rebalancer.getAccountOriginalFactorAllocations(test_account, verbose=True)
print(type(account_factor_allocations))
print(account_factor_allocations)

account_factor_weights = rebalancer.getAccountFactorWeights(test_account, verbose=True)
print(type(account_factor_weights))
# write_weights(account_factor_weights

account_target_factor_allocations = rebalancer.getAccountTargetFactorAllocations(test_account, verbose=True)
print(type(account_target_factor_allocations))
# for factor, allocation in account_target_factor_allocations.items():
#     print(f"{factor}: {allocation}")

account_factor_objective = rebalancer.getAccountFactorObjective(test_account, verbose=True)
print(type(account_factor_objective))

account_turnover_objective = rebalancer.getAccountTurnoverObjective(test_account, verbose=True)
print(type(account_turnover_objective))

account_complexity_objective = rebalancer.getAccountComplexityObjective(test_account, verbose=True)
print(type(account_complexity_objective))

account_constraints = rebalancer.getAccountConstraints(test_account, verbose=True)
print(type(account_constraints))


In [ ]:
# rebalance an account
account_rebalancer = rebalancer.getAccountRebalancer(test_account)
account_rebalancer.rebalance(verbose=True)

account_ticker_results = account_rebalancer.getTickerResults()
print(type(account_ticker_results))
write_table(account_ticker_results, column_formats)

account_factor_results = account_rebalancer.getFactorResults(verbose=True)
print(type(account_factor_results))
write_table(account_factor_results, column_formats)

In [ ]:
# get valid account for testing
# get all available accounts
all_tickers = port.getAccountTickers()
available_accounts = all_tickers.index.get_level_values('Account').unique()

# Print all available accounts
print("Available accounts:")
for account in available_accounts:
    print(f"  - {account}")

# Get the first account name for testing
test_account = available_accounts[0]
print(f"\nUsing account for test: {test_account}")

# get account tickers
account_tickers = port.getAccountTickers(accounts=[test_account])
account_tickers = sorted(account_tickers.index.get_level_values('Ticker').unique().tolist())

# print out the account tickers
print(f"Account tickers for {test_account}:")
for ticker in account_tickers:
    print(f"  - {ticker}")

In [ ]:
# get the cononical factor weights matrix
factor_weights_matrix = port._create_factor_weights_matrix(tickers=account_tickers,
                                                           factors=target_factor_allocations.index,
                                                           verbose=True)

factor_weights_column_formats = {
    'Factor': {'width': 24}
}
for column in factor_weights_matrix.columns:
    factor_weights_column_formats[column] = {'width': 7, 'decimal': 2, 'type': '%'}

#write_table(factor_weights_matrix, factor_weights_column_formats)

In [ ]:
# get variable vectors
variables = port._create_variable_vectors(tickers=account_tickers,
                                          account='my_account',
                                          verbose=True)
#variables

In [ ]:
# create target allocations vector
# results.loc['Bogus Factor'] = {
#     'Original Allocation': 0.0,
#     'New Allocation': 0.0
# }
# print(results)

# calculate account proportion of the total portfolio
# Get account's current allocation as percentage of total portfolio
account_metrics = port.getMetrics('Account', portfolio_allocation=True)
account_proportion = account_metrics.loc[test_account, 'Allocation']
print(f"Account proportion: {account_proportion:,.2%}")

target_allocations = port._create_target_factor_allocations_vector(
    target_allocations=target_factor_allocations,
    account_proportion=account_proportion,
    verbose=True
)
#target_allocations

In [ ]:
# Now you can use this account to create optimization components
account_optimization_components = port._create_account_optimization_components(
    account=test_account,
    target_factor_allocations=results['New Allocation'],
    verbose=True
)
account_optimization_components

In [17]:
ticker_results, factor_results = port.rebalance(results['New Allocation'],
                                                turnover_penalty=0,
                                                complexity_penalty=0,
                                                min_ticker_alloc=0,
                                                verbose=False)


In [ ]:

write_table(ticker_results, column_formats)
total_alloc = ticker_results['New Allocation'].sum()
print(f"Total allocation: {total_alloc:,.2%}")

print()

write_table(factor_results, column_formats)
total_alloc = factor_results['New Allocation'].sum()
print(f"Total allocation: {total_alloc:,.2%}")

In [ ]:
ticker_results, factor_results = port.rebalance_portfolio(results['New Allocation'],
                                                turnover_penalty=0,
                                                complexity_penalty=0,
                                                account_align_penalty=1,
                                                min_ticker_alloc=0,
                                                verbose=True)

In [ ]:
ticker_results_with_totals = ticker_results.copy()
ticker_results_with_totals.loc['ZZZ-TOTAL'] = ticker_results_with_totals.sum()
# Filter to only include rows where both Original and New allocations are non-zero
ticker_results_with_totals = ticker_results_with_totals[
    (ticker_results_with_totals['Original Allocation'] != 0) | 
    (ticker_results_with_totals['New Allocation'] != 0)
]

write_table(ticker_results_with_totals, column_formats)

print()

factor_results_with_totals = factor_results.copy()
factor_results_with_totals.loc['ZZZ-TOTAL'] = factor_results_with_totals.sum()
# Filter to only include rows where both Original and New allocations are non-zero
factor_results_with_totals = factor_results_with_totals[
    (factor_results_with_totals['Original Allocation'] != 0) | 
    (factor_results_with_totals['New Allocation'] != 0)
]
write_table(factor_results_with_totals, column_formats)